# Demonstrate PV Access

- What is PVA?
- View area detector images with `c2dataviewer`
- Python code to receive and process PVA data
- (stretch goal) PVA for diffractometer orientation

## What is PVA?

From the [EPICS Documentation](https://docs.epics-controls.org/en/latest/pv-access/OverviewOfpvData.html):

> pvAccess, pvData and other related modules have been introduced into EPICS to add support for structured data.

The [EPICS web site](https://epics-controls.org/resources-and-support/documents/pvaccess/) says:

> The most prominent new feature of EPICS 7 (and beyond) is the addition of the possibility to manipulate and transport structured data over the network. The data manipulation library is called pvData and the network protocol is called pvAccess.

PVA differs from the well-known Channel Access protocol by providing multiple pieces of information in a defined structure.  In Channel Access (CA), those same multiple pieces of information would be provided by separate (CA) PVs.  Area detector images are a great example.  With PVA, the entire image, its shape, bit depth, color mode, attributes, etc. are communicated in a single PV.

## View PVA images

To view images published as PVA PVs, we'll need:

- software to monitor the PVA PV and view the images
- a PVA PV that is the source of the images to be viewed

### Viewer

The [`c2dataviewer` (`c2dv`)](https://pypi.org/project/c2dataviewer/) was developed for the APS accelerator as part of the APS-U.  It has several modules, one of which is to view images from EPICS area detectors where the image is published as a PV Access PV.  Note that `c2dv` is Python software.

The `c2dv` Python software is installed into its own conda environment, to isolate its version requirements.  That environment was created with these commands:

```bash
conda create -n c2dv -c epics -c conda-forge blosc pvapy
conda activate c2dv
pip install c2dataviewer
```

A bash shell script runner (`c2dv_image.sh`) makes it easier to remember how to start `c2dv`:

```bash
#!/bin/bash

if [ "${#}" != "1" ]; then
    echo "usage:  ${0} PVA_PV"
else
    source $(which activate) c2dv
    c2dv --app image --pv "${1}" &
fi
```

### Area Detector EPICS IOC

Next, we need an area detector IOC.  This demo uses a simulated area detector (1k x 1k, 8bits/pixel, monochrome, prefix=`ad:`), ADSimDetector.  The demo uses the standard simulated image.   (`ad:` is the demo IOC's prefix, use your own here)

### generate images

1. Start generating (simulated) images from the detector. <br />  `caput ad:cam1:Acquire 1`
1. Publish the images over PVA by enabling the detector's PVA plugin: <br />  `caput ad:Pva1:EnableCallbacks 1`
1. Test that the expected image PVA is available: <br />  `pvinfo ad:Pva1:Image`
1. Test that images are available by PVA: <br />  `pvget ad:Pva1:Image`  <br />  (This command will print the entire image as numbers.  Lots of output.  Then other information in the structure is printed.)

In [1]:
! caput ad:cam1:Acquire 1

Old : ad:cam1:Acquire                Acquire
New : ad:cam1:Acquire                Acquire


Publish the images over PVA by enabling the detector's PVA plugin:

In [2]:
! caput ad:Pva1:EnableCallbacks 1

Old : ad:Pva1:EnableCallbacks        Enable
New : ad:Pva1:EnableCallbacks        Enable


Test that the expected image PVA is available:

In [3]:
! pvinfo ad:Pva1:Image

ad:Pva1:Image
Server: 192.168.144.94:5075
Type:
    epics:nt/NTNDArray:1.0
        union value
            boolean[] booleanValue
            byte[] byteValue
            short[] shortValue
            int[] intValue
            long[] longValue
            ubyte[] ubyteValue
            ushort[] ushortValue
            uint[] uintValue
            ulong[] ulongValue
            float[] floatValue
            double[] doubleValue
        codec_t codec
            string name
            any parameters
        long compressedSize
        long uncompressedSize
        dimension_t[] dimension
            dimension_t
                int size
                int offset
                int fullSize
                int binning
                boolean reverse
        int uniqueId
        time_t dataTimeStamp
            long secondsPastEpoch
            int nanoseconds
            int userTag
        epics:nt/NTAttribute:1.0[] attribute
            epics:nt/NTAttribute:1.0
                stri

Test that images are available by PVA.  (The `pvget` command will print the entire image as numbers.  Lots of output. We suppress that here with a pipe to `grep` to remove that `ubyte` content.)  The other information in the structure is printed.

In [4]:
! pvget ad:Pva1:Image | grep -v ubyte

ad:Pva1:Image epics:nt/NTNDArray:1.0 
    union value
    codec_t codec
        string name 
        any parameters
            int  5
    long compressedSize 1048576
    long uncompressedSize 1048576
    dimension_t[] dimension
        dimension_t 
            int size 1024
            int offset 0
            int fullSize 1024
            int binning 1
            boolean reverse false
        dimension_t 
            int size 1024
            int offset 0
            int fullSize 1024
            int binning 1
            boolean reverse false
    int uniqueId 1362424
    time_t dataTimeStamp 2023-10-12 16:52:55.236  
        long secondsPastEpoch 1697147575
        int nanoseconds 235581159
        int userTag 0
    epics:nt/NTAttribute:1.0[] attribute
        epics:nt/NTAttribute:1.0 
            string name ColorMode
            any value
                int  0
            string descriptor "Color mode"
            int sourceType 0
            string source Driver
    string desc

### view images

1. Then, start the viewer:  <br />  `c2dv_image.sh ad:Pva1:Image`
1. Auto-scale the image so that the full 8-bit range of each pixel value is shown.

## Python code to read this PVA

Demonstrate how to read the area detector image in Python using [`pvapy`](https://pypi.org/project/pvapy/).  This demo uses the same PVA PV as above: `ad:Pva1:Image`.

NOTE: For richer examples, use [`pva_examiner`](https://github.com/BCDA-APS/bdp_controls/blob/main/examples/pva_examiner.py).

In [5]:
import datetime
import time

import pvaccess

PVA_PV = "ad:Pva1:Image"
PROTOCOL = pvaccess.PVA

Connect with PV using PVA protocol.

In [6]:
channel = pvaccess.Channel(PVA_PV, PROTOCOL)

Start monitoring for values _after_ connecting.  Add a short delay (empirical) for the connection period.

In [7]:
channel.startMonitor()
time.sleep(0.1)
print(f"{PROTOCOL} {channel.getName()=}  {channel.isConnected() = }")

PVA channel.getName()='ad:Pva1:Image'  channel.isConnected() = True


Get the latest image data from the `channel`.  Then, stop expecting any further PVA monitors.

In [8]:
pv_object = channel.get()
print(f"{pv_object=}")
channel.stopMonitor()

pv_object=<pvaccess.pvaccess.PvObject object at 0x7f602e3728f0>


Add a couple support functions from the [`pva_examiner`](https://github.com/BCDA-APS/bdp_controls/blob/main/examples/pva_examiner.py) online example code.  These functions:

- read any attributes of the PVA PV as a Python dictionary
- read the image payload, if it is found (assumes uint8 data, for this example)

In [9]:
# https://github.com/BCDA-APS/bdp_controls/blob/main/examples/pva_examiner.py

def get_pva_ndattributes(pv_object):
    """Return a dict with the NDAttributes."""
    obj_dict = pv_object.get()
    attributes = {
        attr["name"]: [v for v in attr.get("value", "")]
        for attr in obj_dict.get("attribute", {})
    }
    for k in "codec uniqueId uncompressedSize".split():
        if k in pv_object:
            attributes[k] = pv_object[k]
    return attributes

def pva_to_image(pv_object):
    """Return the image from the PVA object.  Or ``None``."""
    if "dimension" in pv_object:
        shape = [axis["size"] for axis in pv_object["dimension"]]
        image = pv_object["value"][0]["ubyteValue"].reshape(*shape)
    else:
        image = None
    return image

Get the timestamp of the image from EPICS (in the PVA PV).  It is possible to be provided by a couple different names.

In [10]:
dt = datetime.datetime.now()
for key in "dataTimeStamp timeStamp".split():
    if key in pv_object:
        # "PVA"
        timestamp = pv_object[key]["secondsPastEpoch"]
        timestamp += pv_object[key]["nanoseconds"] * 1e-9
        dt = datetime.datetime.fromtimestamp(timestamp)
        break
print(f"{PROTOCOL}: '{PVA_PV}' at {dt}")

PVA: 'ad:Pva1:Image' at 2023-10-12 16:52:55.623944


Show some basic information about this image frame:

In [11]:

image = pva_to_image(pv_object)
if image is not None:
    print(
        f"{image.shape=}"
        f"  {image.dtype=}"
        f"  {image.min()=}"
        f"  {image.max()=}"
    )


image.shape=(1024, 1024)  image.dtype=dtype('uint8')  image.min()=6  image.max()=145


Show the PVA's attributes, if any.

In [12]:

attributes = get_pva_ndattributes(pv_object)

# print by key:value
for i, k in enumerate(attributes, start=1):
    print(f"#{i}  {k}\t{attributes[k]}")

#1  ColorMode	[{'value': 0}, {'value': pvaccess.pvaccess.ScalarType.INT}]
#2  codec	{'name': '', 'parameters': ({'value': 5}, {'value': pvaccess.pvaccess.ScalarType.INT})}
#3  uniqueId	1362495
#4  uncompressedSize	1048576


## PVA for diffractometer orientation

When using a diffractometer to explore the scattering from a single crystal, important steps are necessary to orient a sample of that crystal and align it to the diffractometer.  These steps are recorded in memory of the control software.  Should that software stop for some reason, the orientation is lost and the steps must be repeated to recreate the orientation.

It would be very useful to store the sample information and its orientation otuside of the control software so that it may be restored in such circumstances.  A PVA structure, with its dictionary of attributes is a good container for the necessary sample orientation details.

For this demo, we'll need:

- a PVA server
- control software with an oriented sample
- steps to write the orientation to the PVA

### PVA server

Python code using the [`pvaccess`
package](https://epics.anl.gov/extensions/pvaPy/production/pvaccess.html)
creates the PVA server in this demo.  We assign this name for the PVA:
`demo:orientation`.  For simplicity, we design this PVA with a structure that is
just a string called `"samples"` which will contain a [JSON](https://json.org)
dictionary with sample and orientation details.

NOTE:  We choose JSON to communicate this structure because it is *extendable*.
For a given diffractometer, the number of samples is not always one, the number
of reflections is not always two, and other details such as the number and names
of the motors is not always the same.  A Python dictionary is a good description
of these details.  [JSON is a data format in which to communicate a Python
dictionary](https://stackoverflow.com/questions/33169404#33169622).

In the real-world case, the server should be run independent from the control
software.  Here, the server is run locally and we make up the example
orientation details.

First, create the PvObject locally.  It is *not published yet* as a PVA.

In [13]:
import pvaccess

pv = pvaccess.PvObject({"samples": pvaccess.STRING})

Assign the PV name as the PVA server is created, then start the server.

In [14]:
pvname = "demo:orientation"
pva_server = pvaccess.PvaServer(pvname, pv)

Test (with a bash command) that the server is running and the PVA has the expected structure.

In [15]:
! pvinfo demo:orientation

demo:orientation
Server: 192.168.144.94:47451
Type:
    structure
        string samples



We have not assigned any values, so the content is empty at first.

In [16]:
! pvget demo:orientation

demo:orientation structure 
    string samples 


Write some JSON content to the PV, from the server.

In [17]:
import json

pv["samples"] = json.dumps({"key1": "value1"})

Show that using a shell command:

In [18]:
! pvget demo:orientation

demo:orientation structure 
    string samples "{\"key1\": \"value1\"}"


Simulate a diffractometer control system where a crystal of *vibranium* has been
oriented. The details are provided in a dictionary of *samples* (in case more
than one crystal has been oriented).

In [19]:
samples = {
    "vibranium": dict(
        name="vibranium", 
        lattice=(4.14, 4.14, 4.14, 90, 90, 90),
        reflections=[
            dict(hkl=(1, 0, 0), motors=(dict(th=30, tth=60, chi=0, phi=30)), wavelength=0.5),
            dict(hkl=(0, 1, 0), motors=(dict(th=30, tth=60, chi=90, phi=30)), wavelength=0.5),
        ],
        # UB=[],
    )
}
pv["samples"] = json.dumps(samples)

Look at the contents of that PVA, first on the command line, then in Python code.

In [20]:
! pvget demo:orientation

print("\nThe PVA object in the server:")
print(f"{pv=}")

print("\nThe JSON content in the PVA object, note it's a string")
print(f"{pv['samples']=}")

print("\nInterpret the JSON.  Note that original structure and data types are recovered.")
print(f"{json.loads(pv['samples'])=}")

print("\nFor each sample, show how many reflections, for example.")
for k, details in json.loads(pv['samples']).items():
    print(f"{k}: {len(details['reflections'])} reflections")

demo:orientation structure 
    string samples "{\"vibranium\": {\"name\": \"vibranium\", \"lattice\": [4.14, 4.14, 4.14, 90, 90, 90], \"reflections\": [{\"hkl\": [1, 0, 0], \"motors\": {\"th\": 30, \"tth\": 60, \"chi\": 0, \"phi\": 30}, \"wavelength\": 0.5}, {\"hkl\": [0, 1, 0], \"motors\": {\"th\": 30, \"tth\": 60, \"chi\": 90, \"phi\": 30}, \"wavelength\": 0.5}]}}"

The PVA object in the server:
pv=<pvaccess.pvaccess.PvObject object at 0x7f602d3157b0>

The JSON content in the PVA object, note it's a string
pv['samples']='{"vibranium": {"name": "vibranium", "lattice": [4.14, 4.14, 4.14, 90, 90, 90], "reflections": [{"hkl": [1, 0, 0], "motors": {"th": 30, "tth": 60, "chi": 0, "phi": 30}, "wavelength": 0.5}, {"hkl": [0, 1, 0], "motors": {"th": 30, "tth": 60, "chi": 90, "phi": 30}, "wavelength": 0.5}]}}'

Interpret the JSON.  Note that original structure and data types are recovered.
json.loads(pv['samples'])={'vibranium': {'name': 'vibranium', 'lattice': [4.14, 4.14, 4.14, 90, 90, 90],